In [1]:
import dill
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector
import time
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import json
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [2]:
df1 = None

In [3]:
def event_action(df3):
    target_action = ['sub_car_claim_click', 
                 'sub_car_claim_submit_click',
                 'sub_open_dialog_click', 
                 'sub_custom_question_submit_click', 
                 'sub_call_number_click', 
                 'sub_callback_submit_click', 
                 'sub_submit_success', 
                 'sub_car_request_submit_click'
                ]

    df3['event_action'] = df3['event_action'].apply(lambda x: 1 if x in target_action else 0)
    return df3

In [4]:
def sample_df3(df3, total_rows =  200000, neg_percent = 50, pos_percent = 50):
    df3_pos = df3[df3['event_action'] == 1].sample(int(total_rows / 100 * pos_percent))
    df3_neg = df3[df3['event_action'] == 0].sample(int(total_rows / 100 * neg_percent))
    df3_pos = df3_pos.reset_index()
    df3_neg = df3_neg.reset_index()
    df3_pos = df3_pos.drop('index', axis=1)
    df3_neg = df3_neg.drop('index', axis=1)
    df3 = pd.concat([df3_pos, df3_neg])
    return df3

In [5]:
def ad_campaign(df3):
    try:
        with open('data/utm_c_frec_dict1.json', 'r') as f:
            utm_c_frec_dict = json.load(f)
    except FileNotFoundError:
        print("oh.., looks like its the first time you run it - lil' bit longer then, m8. pls hold:)")

        utm_c_frec_dict = {}
        counter = 1
        for pos in df3.utm_campaign.unique():
            if len(df3[(df3.utm_campaign == pos) & (df3.event_action == 1)]) == 0:
                utm_c_frec_dict[str(pos)] = 0
            else:
                utm_c_frec_dict[str(pos)] = round(len(df3[(df3.utm_campaign == pos) & (df3.event_action == 1)]) / len(df3[df3.utm_campaign == pos]), 5)

            #print(counter)
            counter = counter  + 1
        with open('data/utm_c_frec_dict1.json', 'w') as f: 
            json.dump(utm_c_frec_dict, f)

    finally:
        df3['camp_succ_rate'] = df3.utm_campaign.apply(lambda x: utm_c_frec_dict[str(x)])
    return df3

In [6]:
def day_of_week(df3):
    df3['new_date'] = pd.to_datetime(df3['visit_date'])
    df3['day_of_week'] = df3.new_date.dt.dayofweek
    return df3

In [7]:
def empties(df3):
    df3.loc[df3.utm_source.isna() == True, 'utm_source'] = 'other'
    df3.loc[df3.utm_adcontent.isna() == True, 'utm_adcontent'] = 'Other'
    df3.loc[df3.device_brand.isna() == True, 'device_brand'] = 'other'
    return df3

In [8]:
def resolution_func(df3):
    #resolution
    bounds = []
    df3['resolution'] = df3.device_screen_resolution.apply(lambda x:eval(x.replace('x','*')))
    for device in df3.device_category.unique():
        q25 = df3[df3.device_category == device].resolution.quantile(0.25)
        q75 = df3[df3.device_category == device].resolution.quantile(0.75)
        iqr = q75 - q25
        bounds.append((device, q25 - 1.5 * iqr, q75 + 1.5 * iqr))




    test_list = list(df3.device_screen_resolution)
    test_list2 = list(df3.device_category)

    for i in range(len(test_list)):
        test_list[i] = eval(test_list[i].replace('x','*'))

    tst_l = list(zip(test_list2, test_list))

    resolution = []

    for i in range(len(tst_l)):
        if tst_l[i][0] == bounds[0][0]:
            resolution.append(bounds[0][0]+'_high' if tst_l[i][1] >= bounds[0][2] * 0.7 else (bounds[0][0]+'_medium' if bounds[0][2] * 0.7 > tst_l[i][1] >= bounds[0][2] * 0.1 else bounds[0][0]+'_low'))
        elif tst_l[i][0] == bounds[1][0]:
            resolution.append(bounds[1][0]+'_high' if tst_l[i][1] >= bounds[1][2] * 0.7 else (bounds[1][0]+'_medium' if bounds[1][2] * 0.7 > tst_l[i][1] >= bounds[1][2] * 0.1 else bounds[1][0]+'_low'))
        elif tst_l[i][0] == bounds[2][0]:
            resolution.append(bounds[2][0]+'_high' if tst_l[i][1] >= bounds[2][2] * 0.7 else (bounds[2][0]+'_medium' if bounds[2][2] * 0.7 > tst_l[i][1] >= bounds[2][2] * 0.1 else bounds[2][0]+'_low'))

    df3['device_screen_resolution'] = resolution
    df3 = df3.drop('resolution', axis=1)
    
    return df3
    

In [9]:
def country(df3, trsh = 0.001):
    #geo_country
    country_list = list(df3.geo_country.unique())
    for i in range(len(country_list)):
        country_list[i] = ( len(df3[df3.geo_country == country_list[i]]), country_list[i])
    country_list = sorted(country_list, reverse=True)

#    trsh = 0.0005
    df3_len = len(df3) 
    for item in country_list:
        if len(df3[df3.geo_country == item[1]]) / df3_len >= trsh:
            continue
        else:
            df3.loc[df3.geo_country == item[1], 'geo_country'] = 'some_unimportant_country'
    return df3

In [10]:
def city(df3, trsh = 0.02):
    #geo_city
    city_list = []
    df3_len = len(df3)
    try:
        with open('data/city_list1.txt', 'r') as f:
            for line in f:
                # remove newline character and parentheses
                line = line.rstrip('\n').replace('(', '').replace(')', '').replace("'", '')
                # split on comma and convert each element to correct type
                tuple_elements = [int(e.strip()) if e.strip().isdigit() else e.strip() for e in line.split(',')]
                # create tuple and add to list
                my_tuple = tuple(tuple_elements)
                city_list.append(my_tuple)


    except FileNotFoundError:
        print("oh.., looks like its the first time you run it - lil' bit longer then, m8. pls hold:)")

        city_list = list(df3.geo_city.unique())
        for item in range(len(city_list)):
            city_list[item] =  (len(df3[df3.geo_city == city_list[item]]), city_list[item])
        city_list = sorted(city_list, reverse=True)

        with open('data/city_list1.txt', 'w') as f:
            for t in city_list:
                f.write(str(t) +'\n')



    finally:
#        trsh = 0.0005
        city_list_valid = []
        for item in city_list:
            #print(item[1], ' - ', round(item[0] / df3_len, 4),'%' )
            if round(len(df3[df3.geo_city == item[1]]) / df3_len, 4) >= trsh:
                city_list_valid.append(item[1])
                #print('trsh == 2000 - ', item[0], item[1], round(item[0] / df3_len, 4) >= trsh, ' - appended')

        df3.loc[(~df3['geo_city'].isin(city_list_valid)), 'geo_city'] = 'some_unimportant_city'
    
    return df3

In [11]:
def encode_stuff(df):
    cols_to_encode = ['utm_source', 
                      'utm_medium', 
                      'utm_adcontent', 
                      'device_brand', 
                      'device_category', 
                      'device_screen_resolution', 
                      'device_browser',
                      'utm_campaign',
                      'geo_country',
                      'geo_city'
                     ]
    #encoding
    encoded_features = pd.DataFrame()

    for col in cols_to_encode:

        pre_encoded_df = df[[col]]
        encoder = OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False)
        encoded_array = encoder.fit_transform(pre_encoded_df)
        #feature_names = [f'{col}_{name}' for name in encoder.get_feature_names_out()]
        feature_names = encoder.get_feature_names_out()
        encoded_df = pd.DataFrame(encoded_array, columns=feature_names)

        if len(encoded_features) == 0:
            encoded_features = encoded_df.copy()
        else:
            encoded_features[feature_names] = encoded_df.values


    df = df.join(encoded_features)
    return df

In [12]:
def scale_stuff(df3):
    #scaling
    cols_to_scale = ['visit_number']

    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df3.loc[:,cols_to_scale])
    scaled_feature_names = [f'{name}_scaled' for name in scaler.get_feature_names_out()]
    #scaler.get_feature_names_out()

    scaled_df = pd.DataFrame(scaled_features, columns=scaled_feature_names)

    df3 = df3.join(scaled_df)

    df3 = df3.drop('visit_number', axis=1)
    return df3

In [41]:
def filter_stuff(df3):
    #pre-existing list of columns
    df_columns = [
        'session_id',
        'hit_date',
        'hit_time',
        'hit_number',
        'hit_type',
        'hit_referer',
        'hit_page_path',
        'event_category',
        'event_label',
        'event_value'
    ]
    
    cols_to_encode = [
        'utm_source',
        'utm_medium', 
        'utm_adcontent',
        'device_brand', 
        'device_category', 
        'device_screen_resolution',
        'device_browser',
        'utm_campaign',
        'geo_country',
        'geo_city'
    ]
    #dropping
    cols_to_drop = []
    for col in df_columns:
        cols_to_drop.append(str(col))
    cols_to_drop = cols_to_drop + ['client_id','new_date', 'visit_date', 'utm_keyword', 'device_os', 'device_model', 'visit_time']    
    df3 = df3.drop(cols_to_drop, axis=1)
    df3 = df3.drop(cols_to_encode, axis=1)
    return df3

In [14]:
def check_stuff(df3):
    #checking

    counter = 0
    for feature in df3.columns:
        if df3[feature].dtype != 'O':
            #print(feature, ' - ', df3[feature].dtype)
            counter += 1
        else:
            print(feature)
    print(counter == len(df3.columns))


    #checking 2
    counter = 0
    for feature in df3.columns:
        if len(df3[df3[str(feature)].isna() == True]) != 0:
            print(feature, ' - ', len(df3[df3[str(feature)].isna() == True]))
            counter += 1




    if counter == 0:
        print('vse zaebis", pustukh fi4ei net')    
    print(df3.shape)
    pass

In [15]:
with open('data/best_params_rf_10k_fit.json', 'r') as f:
    best_params = json.load(f)
best_params

{'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 400}

In [16]:
def predict_stuff(df3):
    y = df3['event_action']
    
    df3 = df3.drop('event_action', axis=1)
    
    x_train, x_test, y_train, y_test = train_test_split(df3,y, test_size=0.3)
    
    rf = RandomForestClassifier(n_estimators=400, min_samples_leaf=2, max_features='sqrt')
    rf.fit(x_train, y_train)
    
    predicted_train = rf.predict(x_train)
    predicted_test = rf.predict(x_test)
    
    print('train acc score - ',accuracy_score(y_train, predicted_train))
    print('test acc score - ', accuracy_score(y_test, predicted_test))

    print('train roc score - ',roc_auc_score(y_train, rf.predict_proba(x_train)[:,1]))
    print('test roc score - ',roc_auc_score(y_test, rf.predict_proba(x_test)[:,1]))
    pass
